# To do:

 - Figure out how to signal injuries

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
import warnings
import math         # haversine_km()
import os

import xgboost as xgb
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib
import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

tgt_stat = "PRA"
print('Target Stat:', tgt_stat)

Today's date: 2026-01-03
Target Stat: PRA


In [2]:
%run ./common_utils.ipynb

# ML Functions

In [3]:
def feature_importance(model):
    importance = model.get_score(importance_type='gain')

    # Convert to table
    df_importance = (
        pd.DataFrame({
            'feature': list(importance.keys()),
            'importance': list(importance.values())
        })
        .sort_values(by='importance', ascending=False)
        .reset_index(drop=True)
    )

    df_importance['pct'] = df_importance.importance.cumsum() / df_importance.importance.sum()
    display(df_importance)

    xgb.plot_importance(model)
    plt.show()

In [4]:
def create_baseline_model(df, pred_col, DFS):
    
    train_df, val_df, test_df = DFS

    if pred_col == 'MP':
        print('Minutes Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            'starter', 'bench', 'reserve'
        ]
    else:
        print(f'{pred_col} Stats Model')
        feature_cols = [
            'MP_lst_gm',
            'MP_last_5_avg',
            'MP_last_10_avg',
            f'{pred_col}_last_3_avg', f'{pred_col}_last_5_avg', f'{pred_col}_last_10_avg',
            f'Def_{pred_col}', f'Def_L5_{pred_col}'
        ]
    
    print('Train:', len(train_df), '/ Validation:', len(val_df), '/ Test:', len(test_df))
    
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert to DMatrix (XGBoost internal format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42
    }

    # Train using native XGBoost API with early stopping
    evals = [(dtrain, "train"), (dval, "val")]
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict on test set
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R²:", r2)
    
    return bst

In [29]:
def hyperparam_tuning(DFS, pred_col, n_iter=20, early_stopping_rounds=50):
    """
    Hyperparameter tuning using native XGBoost API and DMatrix,
    with early stopping support (compatible with XGBoost 3.1.2)
    """

    train_df, val_df, test_df = DFS
    feature_cols = [col for col in train_df.columns if col not in ['Date', pred_col]]
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert datasets to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    # Hyperparameter search space
    param_dist = {
        "n_estimators": randint(300, 1500),
        "learning_rate": uniform(0.01, 0.05),
        "max_depth": randint(3, 6),
        "min_child_weight": randint(1, 8),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.7, 0.3),
        "gamma": uniform(0, 2),
        "reg_lambda": uniform(0, 5),
        "reg_alpha": uniform(0, 2)
    }

    # Sample n_iter random parameter combinations
    param_list = []
    for _ in range(n_iter):
        sample = {k: (v.rvs() if hasattr(v, "rvs") else v) for k, v in param_dist.items()}
        sample['n_estimators'] = int(sample['n_estimators'])
        sample['max_depth'] = int(sample['max_depth'])
        sample['min_child_weight'] = int(sample['min_child_weight'])
        param_list.append(sample)

    best_mae = float('inf')
    best_params = None
    best_bst = None

    # Manual hyperparameter search
    for i, params in enumerate(param_list):
        print(f"\nTrial {i+1}/{n_iter}: {params}")
        num_boost_round = params.pop('n_estimators')
        params.update({
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "device": "cuda",
            "seed": 42
        })
        evals = [(dtrain, 'train'), (dval, 'val')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )
        # Predict on validation set to compute MAE
        val_preds = bst.predict(dval, iteration_range=(0, bst.best_iteration))
        mae = mean_absolute_error(y_val, val_preds)
        print(f"Validation MAE: {mae:.4f}")
        if mae < best_mae:
            best_mae = mae
            best_params = params.copy()
            best_bst = bst

    print("\nBest validation MAE:", best_mae)
    print("Best parameters:", best_params)

    # Predict on test set using best model
    preds = best_bst.predict(dtest, iteration_range=(0, best_bst.best_iteration))
    test_df[pred_col] = y_test
    test_df[f'{pred_col}_preds'] = preds
    test_df['Team'] = team_encoder.inverse_transform(test_df["Team"])
    test_df['Opp'] = team_encoder.inverse_transform(test_df["Opp"])
    test_df['Player'] = player_encoder.inverse_transform(test_df["Player"])
    test_df['Pos'] = position_encoder.inverse_transform(test_df["Pos"])
    analyze_df = test_df[['Date', 'Team', 'Player', 'Pos', 'Opp', pred_col, f'{pred_col}_preds']]
    print("\nTest Metrics:")
    print("RMSE:", np.sqrt(mean_squared_error(y_test, preds)))
    print("MAE:", mean_absolute_error(y_test, preds))
    print("R²:", r2_score(y_test, preds))

    return best_bst, preds, y_test, analyze_df

### Create Base df

In [26]:
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for i in [2022, 2023, 2024, 2025]:
    df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
    df_actuals['Season'] = i
    df = pd.concat([df, df_actuals])

    df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
    df_schd['Season'] = i
    df2 = pd.concat([df2, df_schd])
    
    df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
    df_gms['Season'] = i
    df3 = pd.concat([df3, df_gms])
    
    df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
    df_inj['Season'] = i
    df4 = pd.concat([df4, df_inj])

df['Date'] = pd.to_datetime(df.Date)
df2['Date'] = pd.to_datetime(df2.Date)
df3['Date'] = pd.to_datetime(df3.Date)
df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
df4['Date'] = pd.to_datetime(df4.Date)

df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
df3_temp['RA'] = df3_temp.REB + df3_temp.AST
df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch['Team_type'] = 'Away'
df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch2['Team_type'] = 'Home'
df_mtch = pd.concat([df_mtch, df_mtch2])
df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch = df_mtch.sort_values(["Team", "Date"])
df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

team_encoder = LabelEncoder()
player_encoder = LabelEncoder()
team_type_encoder = LabelEncoder()
position_encoder = LabelEncoder()
status_encoder = LabelEncoder()

# Encode string cols
team_encoder.fit(pd.concat([df["Team"], df["Opp"]], axis=0))
df["Team"] = team_encoder.transform(df["Team"])
df["Opp"] = team_encoder.transform(df["Opp"])
df["Player_name"] = df.Player
df["Player"] = player_encoder.fit_transform(df["Player"])
df["Pos"] = position_encoder.fit_transform(df["Pos"])
df['Team_type'] = team_type_encoder.fit_transform(df['Team_type'])
df["Status"] = status_encoder.fit_transform(df["Status"])
df_pred = df.copy()
df = df[(df.Active == 1) & (df.MP > 0)].sort_values(['Season', 'Date', 'Team', 'Player']).reset_index(drop=True)
print('base df created', datetime.now())

base df created 2026-01-03 12:44:35.927737


# Minutes Projection Model

In [7]:
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_km_from_row(row):
    prev = row['PrevLocation']
    cur  = row['Location']
    # missing prev => first game => no travel
    if pd.isna(prev) or pd.isna(cur):
        return 0.0
    # same arena => 0
    if prev == cur:
        return 0.0
    # lookup coords
    prev_coords = arenas.get(prev)
    cur_coords  = arenas.get(cur)
    if not prev_coords or not cur_coords:
        # fallback if code not found
        return 0.0
    return haversine_km(prev_coords[0], prev_coords[1], cur_coords[0], cur_coords[1])

In [30]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP', 'TOV', 'PF', '+/-',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'cup_gm', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP', 'TOV', 'PF', '+/-']:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )
        df[f"{col}_change_L1"] = df[f"{col}_lst_gm"] - df[f"{col}_last_5_avg"]
        df[f"{col}_change_L3"] = df[f"{col}_last_3_avg"] - df[f"{col}_last_10_avg"]
        df[f"{col}_pct_change"] = (
            (df[f"{col}_lst_gm"] - df[f"{col}_last_10_avg"]) /
            (df[f"{col}_last_10_avg"] + 1e-6)
        )
    df["MP_spike"] = (df["MP_lst_gm"] > df["MP_last_10_avg"] + 8).astype(int)
    df["MP_drop"]  = (df["MP_lst_gm"] < df["MP_last_10_avg"] - 8).astype(int)
    df["MP_trend"] = df["MP_last_3_avg"] - df["MP_last_10_avg"]

    games_last_7_days = df.groupby(['Player', 'Season']).rolling('7D', on='Date')['MP'].count().shift(1).to_frame(name='games_last_7_days').reset_index()
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['games_last_7_days'] = df.games_last_7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['blowout'] = np.where(abs(df.Spread >= 15), 1, 0)
    
    # Location based features
    df["PrevOpp"] = df.groupby("Player")["Opp"].shift(1)
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['travel_km'] = df.apply(travel_km_from_row, axis=1).fillna(0)
    df['travel_hours'] = df['travel_km'] / 800.0      # approximate flight hours
    df['is_long_trip'] = (df['travel_km'] > 1500).astype(int)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)
    
    df = df.drop(['Season', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'Szn_Wins', 'TOV', 'PF', '+/-', 
                  'PrevOpp', 'PrevLocation', 'Location'], axis=1)    
    
    return df

In [32]:
df_mins = df.copy()
df_mins = setup_df_mins(con, df_mins)
display(df_mins)

n = len(df_mins)
train_end = int(0.8 * n)
val_end   = int(0.9 * n)
mins_train_df = df_mins.iloc[:train_end]
mins_val_df   = df_mins.iloc[train_end:val_end]
mins_test_df  = df_mins.iloc[val_end:]
mins_DFS = (mins_train_df, mins_val_df, mins_test_df)

# mins_model = create_baseline_model(df_mins, "MP", mins_DFS)
mins_model, mins_preds, y_test_mins, analyze_df_mins = hyperparam_tuning(mins_DFS, "MP", n_iter=1)
# feature_importance(mins_model)

# mins_model.save_model("../ML_models/mins_model.json")
# print('Saved minutes model!')

,Date,Team,Team_type,Opp,Player,Pos,B2B,MP,Spread,Total,team_game_num,cup_gm,pstszn_gm,is_OT,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,MP_change_L1,MP_change_L3,MP_pct_change,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,TOV_change_L1,TOV_change_L3,TOV_pct_change,PF_lst_gm,PF_last_3_avg,PF_last_3_std,PF_last_5_avg,PF_last_5_std,PF_last_10_avg,PF_last_10_std,PF_change_L1,PF_change_L3,PF_pct_change,+/-_lst_gm,+/-_last_3_avg,+/-_last_3_std,+/-_last_5_avg,+/-_last_5_std,+/-_last_10_avg,+/-_last_10_std,+/-_change_L1,+/-_change_L3,+/-_pct_change,MP_spike,MP_drop,MP_trend,games_last_7_days,prev_team_mins_pct,role,missed_games,blowout,DaysLstGm,travel_km,travel_hours,is_long_trip,same_arena
0,2022-10-18,1,1,22,15,0,0,23.10,9.0,243.0,1,0,0,0,NaN,23.850000,NaN,22.776,NaN,23.716,NaN,NaN,0.134000,NaN,NaN,1.666667,NaN,1.6,NaN,1.2,NaN,NaN,0.466667,NaN,NaN,2.333333,NaN,3.0,NaN,2.5,NaN,NaN,-0.166667,NaN,NaN,8.666667,NaN,8.2,NaN,10.4,NaN,NaN,-1.733333,NaN,0,0,0.134000,4,NaN,1,0,0,0,0.0,0.0,0,0
1,2022-10-18,1,1,22,61,0,0,8.28,9.0,243.0,1,0,0,0,NaN,5.410000,NaN,3.964,NaN,6.374,NaN,NaN,-0.964000,NaN,NaN,0.333333,NaN,0.2,NaN,0.3,NaN,NaN,0.033333,NaN,NaN,0.333333,NaN,0.2,NaN,1.0,NaN,NaN,-0.666667,NaN,NaN,-2.333333,NaN,-1.0,NaN,-3.0,NaN,NaN,0.666667,NaN,0,0,-0.964000,2,NaN,2,0,0,0,0.0,0.0,0,0
2,2022-10-18,1,1,22,189,4,0,24.03,9.0,243.0,1,0,0,0,NaN,11.733333,NaN,13.474,NaN,12.434,NaN,NaN,-0.700667,NaN,NaN,2.000000,NaN,1.2,NaN,1.0,NaN,NaN,1.000000,NaN,NaN,0.333333,NaN,0.4,NaN,0.3,NaN,NaN,0.033333,NaN,NaN,-1.000000,NaN,-0.2,NaN,-1.3,NaN,NaN,0.300000,NaN,0,0,-0.700667,3,NaN,1,0,0,0,0.0,0.0,0,0
3,2022-10-18,1,1,22,257,1,0,23.95,9.0,243.0,1,0,0,0,NaN,14.046667,NaN,14.498,NaN,14.405,NaN,NaN,-0.358333,NaN,NaN,0.333333,NaN,0.4,NaN,0.6,NaN,NaN,-0.266667,NaN,NaN,1.333333,NaN,1.4,NaN,1.3,NaN,NaN,0.033333,NaN,NaN,-1.666667,NaN,-3.0,NaN,-4.5,NaN,NaN,2.833333,NaN,0,0,-0.358333,4,NaN,1,0,0,0,0.0,0.0,0,0
4,2022-10-18,1,1,22,357,3,0,38.57,9.0,243.0,1,0,0,0,NaN,15.055000,NaN,15.055,NaN,15.055,NaN,NaN,0.000000,NaN,NaN,1.500000,NaN,1.5,NaN,1.5,NaN,NaN,0.000000,NaN,NaN,1.000000,NaN,1.0,NaN,1.0,NaN,NaN,0.000000,NaN,NaN,-8.000000,NaN,-8.0,NaN,-8.0,NaN,NaN,0.000000,NaN,0,0,0.000000,2,NaN,1,0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84329,2026-01-02,29,1,2,428,3,1,24.60,20.0,218.0,33,0,0,0,32.55,28.206667,5.046844,25.258,4.061203,23.974,9.332733,7.292,4.232667,0.357721,0.0,0.666667,1.00000,0.4,0.836660,0.4,1.100505,-0.4,0.266667,-0.999998,1.0,2.333333,1.000000,2.4,1.870829,2.2,1.581139,-1.4,0.133333,-0.545454,4.0,-1.000000,10.440307,0.0,8.043631,1.0,9.046178,4.0,-2.000000,2.999997,1,0,4.232667,4,0.135625,2,0,1,2,0.0,0.0,0,0
84330,2026-01-02,29,1,2,476,3,1,22.83,20.0,218.0,33,0,0,0,22.63,21.520000,5.676067,21.706,4.818207,24.625,8.425520,0.924,-3.105000,-0.081015,0.0,0.666667,0.57735,0.6,0.836660,1.5,1.100505,-0.6,-0.833333,-0.999999,5.0,2.333333,2.645751,2.2,2.302173,2.4,1.911951,2.8,-0.066667,1.083333,-7.0,3.000000,8.621678,-0.6,8.348653,-0.9,8.941787,-6.4,3.900000,6.777785,0,0,-3.105000,3,0.094292,2,0,1,2,0.0,0.0,0,0
84331,2026-01-02,29,1,2,542,1,1,16.87,20.0,218.0,33,0,0,0,19.62,21.150000,6.765740,21.236,5.928336,23.119,5.274862,-1.616,-1.969000,-0.151347,1.0,0.666667,0.57735,0.4,0.836660,1.1,1.032796,0.6,-0.433333,-0.090909,2.0,1.000000,2.081666,1.4,1.870829,2.4,1.885618,0.6,-1.400000,-0.166667,6.0,5.666667,7.000000,3.8,8.700575,3.7,9.046178,2.2,1.966667,0.621621,0,0,-1.969000,4,0.081750,2,0,1,2,0.0,0.0,0,0
84332,2026-01-02,29,1,2,739,4,1,28.00,20.0,218.0,33,0,0,0,28.37,26.340000,4.445413,24.012,5.809890,22.344,5.360078,4.358,3.996000,0.269692,1.0,1.666667,0.57735,1.4,0.547723,1.2,0.948683,-0.4,0.466667,-0.166667,1.0,1.


Trial 1/1: {'n_estimators': 1159, 'learning_rate': np.float64(0.03055965768707694), 'max_depth': 4, 'min_child_weight': 4, 'subsample': np.float64(0.7830743888965156), 'colsample_bytree': np.float64(0.9503188259392898), 'gamma': np.float64(1.1842329808288412), 'reg_lambda': np.float64(0.13830780536896514), 'reg_alpha': np.float64(1.799634225288872)}
Validation MAE: 4.8007

Best validation MAE: 4.8006641548977305
Best parameters: {'learning_rate': np.float64(0.03055965768707694), 'max_depth': 4, 'min_child_weight': 4, 'subsample': np.float64(0.7830743888965156), 'colsample_bytree': np.float64(0.9503188259392898), 'gamma': np.float64(1.1842329808288412), 'reg_lambda': np.float64(0.13830780536896514), 'reg_alpha': np.float64(1.799634225288872), 'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 5.71456516593874
MAE: 4.279622185978246
R²: 0.6816494942865389
Saved minutes model!


In [10]:
rmse = np.sqrt(mean_squared_error(y_test_mins, mins_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_mins, mins_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', 'MP_last_5_avg']][df_yesterday.MP > 0]

df_yesterday['Diff'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff'] <= rmse, 1, 0)

print("\nYesterday's Results:")
print("Total Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), '/', df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff'], axis=1)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 5.735304749503843

Yesterday's Results:
Total Accuracy (InRMSE_Range): 0.8571428571428571
66 / 77


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
0,2026-01-02,MIL,Giannis Antetokounmpo,PF,CHO,28.056765,29.83,27.348823,1
15,2026-01-02,MIL,Kevin Porter Jr.,PG,CHO,36.956581,36.02,34.190373,1
41,2026-01-02,MIL,Ryan Rollins,PG,CHO,31.936579,36.95,28.443233,1
44,2026-01-02,MIL,Myles Turner,C,CHO,29.648409,22.40,28.092374,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
1,2026-01-02,NOP,Zion Williamson,PF,POR,29.330368,30.70,27.323755,1
11,2026-01-02,NOP,Jose Alvarado,PG,POR,18.476370,20.50,22.878635,1
29,2026-01-02,NOP,Jordan Poole,PG,POR,26.382700,29.80,27.893039,1
31,2026-01-02,NOP,Yves Missi,C,POR,17.977770,20.95,13.251665,1
59,2026-01-02,NOP,Jeremiah Fears,PG,POR,26.899406,29.17,26.541997,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
2,2026-01-02,ATL,Onyeka Okongwu,C,NYK,33.483395,29.77,33.891344,1
9,2026-01-02,ATL,Dyson Daniels,SG,NYK,34.442986,37.52,33.834845,1
14,2026-01-02,ATL,Jalen Johnson,SF,NYK,35.602341,38.00,35.286580,1
43,2026-01-02,ATL,Nickeil Alexander-Walker,SG,NYK,32.874153,34.32,32.271518,1
52,2026-01-02,ATL,Kristaps Porzingis,C,NYK,24.809210,17.27,24.157474,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
3,2026-01-02,CLE,Donovan Mitchell,SG,DEN,31.797260,35.12,32.151575,1
28,2026-01-02,CLE,Evan Mobley,PF,DEN,29.077856,32.28,31.352750,1
50,2026-01-02,CLE,Darius Garland,PG,DEN,30.608122,32.15,31.261514,1


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
27,2026-01-02,NYK,Mikal Bridges,SF,ATL,36.124950,35.15,36.024580,1
47,2026-01-02,NYK,Tyler Kolek,PG,ATL,17.961246,17.23,21.082893,1
48,2026-01-02,NYK,Jalen Brunson,PG,ATL,36.169998,34.90,36.665343,1
73,2026-01-02,NYK,OG Anunoby,PF,ATL,36.486786,34.98,33.670530,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
33,2026-01-02,POR,Donovan Clingan,C,NOP,27.523264,29.52,28.304173,1
35,2026-01-02,POR,Deni Avdija,SF,NOP,36.620758,38.40,35.301018,1
62,2026-01-02,POR,Toumani Camara,PF,NOP,33.552998,33.73,32.072895,1
74,2026-01-02,POR,Shaedon Sharpe,SG,NOP,32.689323,30.97,31.564539,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
34,2026-01-02,IND,Andrew Nembhard,PG,SAS,32.780666,35.55,31.469472,1
38,2026-01-02,IND,Bennedict Mathurin,SF,SAS,33.717758,19.77,31.400565,0
40,2026-01-02,IND,Pascal Siakam,PF,SAS,33.309406,35.87,31.635262,1
53,2026-01-02,IND,Jay Huff,C,SAS,17.100142,19.57,22.033743,1
64,2026-01-02,IND,Aaron Nesmith,SF,SAS,27.858086,29.08,24.372898,1
76,2026-01-02,IND,T.J. McConnell,PG,SAS,17.346643,18.15,16.627263,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
54,2026-01-02,WAS,Bilal Coulibaly,SG,BRK,27.729412,29.17,26.967677,1
60,2026-01-02,WAS,CJ McCollum,SG,BRK,32.936527,31.12,32.299628,1
77,2026-01-02,WAS,Khris Middleton,SF,BRK,22.256313,22.83,22.106077,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,MP_last_5_avg,InRMSE_Range
57,2026-01-02,GSW,Brandin Podziemski,SG,OKC,26.319727,26.3,28.978905,1


# Main Model

In [11]:
def setup_df_main(df):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = ['PTS']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', f'Def_{tgt_stat}', f'Def_L5_{tgt_stat}',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA', tgt_stat] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

In [35]:
df_main = df.copy()
df_main = setup_df_main(df_main)
display(df_main)

n = len(df_main)
train_end = int(0.65 * n)
val_end   = int(0.85 * n)
main_train_df = df_main.iloc[:train_end]
main_val_df   = df_main.iloc[train_end:val_end]
main_test_df  = df_main.iloc[val_end:]
main_DFS = (main_train_df, main_val_df, main_test_df)

# stat_model = create_baseline_model(df_main, tgt_stat, main_DFS)
stat_model, stat_preds, y_test_stat, analyze_df_stat = hyperparam_tuning(main_DFS, tgt_stat, n_iter=1)
# feature_importance(stat_model)

# stat_model.save_model(f"../ML_models/{tgt_stat}_model.json")
# print(f'Saved {tgt_stat} model!')

,Date,Team,Opp,Player,Pos,MP,PRA,Def_PRA,Def_L5_PRA,Spread,Total,is_OT,usage_proxy_L3,usage_proxy_L5,usage_proxy_L10,usage_proxy_weighted,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,FGA_lst_gm,FGA_last_3_avg,FGA_last_3_std,FGA_last_5_avg,FGA_last_5_std,FGA_last_10_avg,FGA_last_10_std,TPA_lst_gm,TPA_last_3_avg,TPA_last_3_std,TPA_last_5_avg,TPA_last_5_std,TPA_last_10_avg,TPA_last_10_std,FTA_lst_gm,FTA_last_3_avg,FTA_last_3_std,FTA_last_5_avg,FTA_last_5_std,FTA_last_10_avg,FTA_last_10_std,PRA_lst_gm,PRA_last_3_avg,PRA_last_3_std,PRA_last_5_avg,PRA_last_5_std,PRA_last_10_avg,PRA_last_10_std,PTS_lst_gm,PTS_last_3_avg,PTS_last_3_std,PTS_last_5_avg,PTS_last_5_std,PTS_last_10_avg,PTS_last_10_std,REB_lst_gm,REB_last_3_avg,REB_last_3_std,REB_last_5_avg,REB_last_5_std,REB_last_10_avg,REB_last_10_std,AST_lst_gm,AST_last_3_avg,AST_last_3_std,AST_last_5_avg,AST_last_5_std,AST_last_10_avg,AST_last_10_std,TOV_lst_gm,TOV_last_3_avg,TOV_last_3_std,TOV_last_5_avg,TOV_last_5_std,TOV_last_10_avg,TOV_last_10_std,role
0,2022-10-18,1,22,15,0,23.10,12.0,16.000000,16.0,9.0,243.0,0,0.340850,0.427717,0.394773,0.372302,NaN,23.850000,NaN,22.776,NaN,23.716,NaN,NaN,6.333333,NaN,8.2,NaN,8.0,NaN,NaN,1.666667,NaN,2.4,NaN,2.4,NaN,NaN,4.000000,NaN,3.2,NaN,2.9,NaN,NaN,20.000000,NaN,20.8,NaN,19.7,NaN,NaN,12.666667,NaN,13.4,NaN,12.2,NaN,NaN,3.000000,NaN,3.8,NaN,3.8,NaN,NaN,4.333333,NaN,3.6,NaN,3.7,NaN,NaN,1.666667,NaN,1.6,NaN,1.2,NaN,1
1,2022-10-18,1,22,61,0,8.28,7.0,16.000000,16.0,9.0,243.0,0,0.302402,0.286831,0.167386,0.284229,NaN,5.410000,NaN,3.964,NaN,6.374,NaN,NaN,1.333333,NaN,0.8,NaN,0.6,NaN,NaN,0.000000,NaN,0.0,NaN,0.0,NaN,NaN,0.000000,NaN,0.4,NaN,0.2,NaN,NaN,2.666667,NaN,2.0,NaN,1.9,NaN,NaN,2.000000,NaN,1.4,NaN,0.9,NaN,NaN,0.333333,NaN,0.4,NaN,0.7,NaN,NaN,0.333333,NaN,0.2,NaN,0.3,NaN,NaN,0.333333,NaN,0.2,NaN,0.3,NaN,2
2,2022-10-18,1,22,189,4,24.03,9.0,9.000000,9.0,9.0,243.0,0,0.321647,0.371026,0.392727,0.343569,NaN,11.733333,NaN,13.474,NaN,12.434,NaN,NaN,3.666667,NaN,4.8,NaN,4.7,NaN,NaN,0.333333,NaN,0.8,NaN,1.1,NaN,NaN,0.000000,NaN,0.6,NaN,0.6,NaN,NaN,7.333333,NaN,11.2,NaN,10.2,NaN,NaN,4.000000,NaN,6.0,NaN,5.8,NaN,NaN,1.000000,NaN,2.2,NaN,1.6,NaN,NaN,2.333333,NaN,3.0,NaN,2.8,NaN,NaN,2.000000,NaN,1.2,NaN,1.0,NaN,1
3,2022-10-18,1,22,257,1,23.95,17.0,17.000000,17.0,9.0,243.0,0,0.354274,0.364454,0.379668,0.359868,NaN,14.046667,NaN,14.498,NaN,14.405,NaN,NaN,4.333333,NaN,4.6,NaN,5.0,NaN,NaN,2.666667,NaN,2.4,NaN,2.7,NaN,NaN,2.000000,NaN,1.8,NaN,1.1,NaN,NaN,7.666667,NaN,7.8,NaN,7.1,NaN,NaN,6.333333,NaN,5.2,NaN,5.0,NaN,NaN,1.000000,NaN,1.6,NaN,1.4,NaN,NaN,0.333333,NaN,1.0,NaN,0.7,NaN,NaN,0.333333,NaN,0.4,NaN,0.6,NaN,1
4,2022-10-18,1,22,357,3,38.57,40.0,91.000000,91.0,9.0,243.0,0,0.322357,0.322357,0.322357,0.322357,NaN,15.055000,NaN,15.055,NaN,15.055,NaN,NaN,3.500000,NaN,3.5,NaN,3.5,NaN,NaN,1.000000,NaN,1.0,NaN,1.0,NaN,NaN,1.000000,NaN,1.0,NaN,1.0,NaN,NaN,10.000000,NaN,10.0,NaN,10.0,NaN,NaN,6.500000,NaN,6.5,NaN,6.5,NaN,NaN,3.000000,NaN,3.0,NaN,3.0,NaN,NaN,0.500000,NaN,0.5,NaN,0.5,NaN,NaN,1.500000,NaN,1.5,NaN,1.5,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84329,2026-01-02,29,2,428,3,24.60,27.0,42.000000,37.0,20.0,218.0,0,0.352183,0.319251,0.310088,0.338094,32.55,28.206667,5.046844,25.258,4.061203,23.974,9.332733,9.0,8.666667,3.214550,7.4,3.420526,6.9,5.363457,3.0,3.666667,4.041452,2.8,3.563706,2.8,3.169297,2.0,2.666667,0.577350,1.6,3.033150,1.6,2.766867,23.0,20.666667,3.511885,19.6,6.220932,17.8,10.863292,12.0,11.000000,4.163332,10.2,4.690416,8.1,7.355270,9.0,9.000000,2.645751,8.6,3.033150,8.7,3.977716,2.0,0.666667,1.527525,0.8,1.414214,1.0,2.000000,0.0,0.666667,1.00000,0.4,0.836660,0.4,1.100505,2
84330,2026-01-02,29,2,476,3,22.83,15.0,


Trial 1/1: {'n_estimators': 1467, 'learning_rate': np.float64(0.017532778114919587), 'max_depth': 5, 'min_child_weight': 2, 'subsample': np.float64(0.9778188722128454), 'colsample_bytree': np.float64(0.8483368309965921), 'gamma': np.float64(1.6478042564429707), 'reg_lambda': np.float64(3.6527866946952248), 'reg_alpha': np.float64(1.7773954705969324)}
Validation MAE: 4.1713

Best validation MAE: 4.171281171158852
Best parameters: {'learning_rate': np.float64(0.017532778114919587), 'max_depth': 5, 'min_child_weight': 2, 'subsample': np.float64(0.9778188722128454), 'colsample_bytree': np.float64(0.8483368309965921), 'gamma': np.float64(1.6478042564429707), 'reg_lambda': np.float64(3.6527866946952248), 'reg_alpha': np.float64(1.7773954705969324), 'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 5.696320203668774
MAE: 4.293236988409264
R²: 0.7919046319280848
Saved PRA model!


In [28]:
# analyze_df_stat = main_test_df.drop([tgt_stat, f'{tgt_stat}_preds'], axis=1)\
#                 .merge(analyze_df_stat[['Date', 'Team', 'Player', tgt_stat, f'{tgt_stat}_preds']], on=['Date', 'Team', 'Player'])
# analyze_df_stat['Diff'] = analyze_df_stat[tgt_stat] - analyze_df_stat[f'{tgt_stat}_preds']
# analyze_df_stat[analyze_df_stat.MP > 38].sort_values('Diff', ascending=True).head(15)

,Date,Team,Opp,Player,Pos,MP,Def_PTS,Def_L5_PTS,Spread,Total,is_OT,three_rate_L3,ft_rate_L3,eFG_L3,TS_L3,usage_proxy_L3,three_rate_L5,ft_rate_L5,eFG_L5,TS_L5,usage_proxy_L5,three_rate_L10,ft_rate_L10,eFG_L10,TS_L10,usage_proxy_L10,three_rate_weighted,ft_rate_weighted,eFG_weighted,TS_weighted,usage_proxy_weighted,MP_lst_gm,MP_last_3_avg,MP_last_3_std,MP_last_5_avg,MP_last_5_std,MP_last_10_avg,MP_last_10_std,FGA_lst_gm,FGA_last_3_avg,FGA_last_3_std,FGA_last_5_avg,FGA_last_5_std,FGA_last_10_avg,FGA_last_10_std,TPA_lst_gm,TPA_last_3_avg,TPA_last_3_std,TPA_last_5_avg,TPA_last_5_std,TPA_last_10_avg,TPA_last_10_std,FTA_lst_gm,FTA_last_3_avg,FTA_last_3_std,FTA_last_5_avg,FTA_last_5_std,FTA_last_10_avg,FTA_last_10_std,PTS_lst_gm,PTS_last_3_avg,PTS_last_3_std,PTS_last_5_avg,PTS_last_5_std,PTS_last_10_avg,PTS_last_10_std,role,PTS,PTS_preds,Diff
11686,2025-12-27,MIL,CHI,Kevin Porter Jr.,PG,38.98,28.307692,42.00,9.0,215.0,0,0.408949,0.548416,0.517044,0.580162,0.515299,0.388959,0.437597,0.429884,0.488485,0.507591,0.388251,0.421501,0.562319,0.621015,0.499253,0.400882,0.502479,0.495423,0.556744,0.511382,33.20,36.113333,15.102438,35.642000,13.144503,34.950000,11.524010,15.0,15.000000,8.144528,15.200000,7.120393,14.800000,6.292853,7.0,6.333333,4.041452,6.000000,3.209361,5.900000,3.687818,9.0,8.000000,4.725816,6.6,3.781534,6.1,2.907844,16.0,21.333333,8.082904,18.2,6.767570,21.7,8.208397,1,8.0,26.060570,-18.060570
8717,2025-12-02,NYK,BOS,Jalen Brunson,PG,39.22,25.312500,27.60,-6.0,240.0,0,0.271930,0.305973,0.514829,0.566411,0.722496,0.343609,0.272306,0.563784,0.612314,0.686039,0.359142,0.278386,0.559474,0.607156,0.670780,0.302155,0.293114,0.533980,0.584256,0.706388,33.73,35.593333,15.328227,35.268000,11.110030,34.556000,9.300263,19.0,22.666667,7.000000,21.600000,5.069517,20.700000,4.532598,6.0,6.000000,1.732051,7.200000,1.303840,7.100000,2.869379,5.0,6.666667,2.886751,5.8,2.190890,5.8,3.071373,18.0,29.333333,5.507571,29.6,5.770615,28.1,5.291503,1,15.0,32.563641,-17.563641
3117,2025-10-25,PHI,CHO,VJ Edgecombe,SG,39.60,40.000000,40.00,4.0,246.0,0,0.500000,0.230769,0.596154,0.593575,0.678673,0.500000,0.230769,0.596154,0.593575,0.678673,0.500000,0.230769,0.596154,0.593575,0.678673,0.500000,0.230769,0.596154,0.593575,0.678673,42.20,42.200000,5.885261,42.200000,17.232372,42.200000,14.818652,26.0,26.000000,9.865766,26.000000,11.541230,26.000000,9.480389,13.0,13.000000,4.000000,13.000000,4.827007,13.000000,4.371626,6.0,6.000000,2.000000,6.0,2.966479,6.0,3.865805,34.0,34.000000,15.874508,34.0,17.239490,34.0,14.273519,1,15.0,32.246395,-17.246395
2168,2025-05-26,MIN,OKC,Anthony Edwards,SG,40.93,27.183673,42.60,-2.0,254.0,0,0.477376,0.346908,0.611237,0.641648,0.600132,0.511187,0.263383,0.640075,0.657936,0.571395,0.460463,0.360547,0.561782,0.597970,0.585480,0.485828,0.323214,0.614943,0.642167,0.590046,29.70,35.446667,3.691021,36.154000,7.683709,38.754000,13.967116,17.0,18.666667,7.023769,18.400000,6.797058,19.700000,8.962267,8.0,8.333333,4.163332,9.000000,3.033150,8.900000,3.984693,2.0,6.333333,1.154701,4.8,1.414214,7.0,2.110819,30.0,26.666667,12.055428,26.4,10.382678,26.9,12.547244,1,16.0,33.103745,-17.103745
10773,2025-12-21,ATL,CHI,Dyson Daniels,SG,39.07,22.040000,28.60,-2.0,302.0,0,0.209691,0.193818,0.522556,0.519140,0.508394,0.208315,0.326291,0.483534,0.512187,0.484526,0.176685,0.239794,0.496849,0.508455,0.422523,0.205977,0.238157,0.508279,0.515986,0.492646,25.85,32.040000,4.388409,33.118000,3.668342,35.226000,6.525779,14.0,15.000000,4.509250,14.200000,3.507136,13.400000,4.001389,5.0,3.000000,1.527525,3.000000,1.788854,2.500000,1.763834,2.0,2.666667,1.000000,4.0,1.483240,2.9,1.173788,11.0,17.333333,4.725816,16.4,4.159327,14.9,4.613988,1,8.0,25.084949,-17.084949
9474,2025-12-07,LAL,PHI,Austin Reaves,SG,39.13,28.619048,23.80,4.0,220.0,0,0.517196,0.580688,0.648810,0.715542,0.604655,0.510317,0.628413,0.735952,0.792469,0.556912,0.454022,0.613003,0.685423,0.750524,0.539471,0.508815,0.598237,0.678614,0.742118,0.583814,33.32,35.830000,7.681154,37.688000,7.805798

In [19]:
rmse = np.sqrt(mean_squared_error(y_test_stat, stat_preds)) # splits[5] = y_test
mae = mean_absolute_error(y_test_stat, stat_preds)
print('RMSE:', rmse)

df_yesterday = pd.read_csv(f'../tables/2025/gmday_preds_{tgt_stat}.csv')
df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
                .rename(columns={"MP": "MP_proj"})

df_gms = pd.read_csv(f"../tables/2025/season_gamelogs.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)
df_gms = df_gms.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df_gms['STL_BLK'] = df_gms.STL + df_gms.BLK
df_gms['PR'] = df_gms.PTS + df_gms.REB 
df_gms['PA'] = df_gms.PTS + df_gms.AST
df_gms['RA'] = df_gms.REB + df_gms.AST
df_gms['PRA'] = df_gms.PTS + df_gms.REB + df_gms.AST

df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', tgt_stat, 'MP']], on=['Date', 'Team', 'Player'])
df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'Pos', 'Opp', 'MP_proj', 'MP', f'{tgt_stat}_line', f'{tgt_stat}_proj', tgt_stat]][df_yesterday.MP > 0]

df_yesterday['Diff'] = df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[f'{tgt_stat}_line']
df_yesterday['Diff2'] = abs(df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[tgt_stat])
df_yesterday['Act_Res'] = np.where(df_yesterday[tgt_stat] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['Pred_Res'] = np.where(df_yesterday[f'{tgt_stat}_proj'] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
df_yesterday['ParlayHit'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff2'] <= rmse, 1, 0)

print("Total Accuracy (ParlayHit):", ((df_yesterday.ParlayHit == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.ParlayHit == 1).sum(), "/", df_yesterday.shape[0])

print("\nTotal Accuracy (InRMSE_Range):", ((df_yesterday.InRMSE_Range == 1).sum() / df_yesterday.shape[0]))
print((df_yesterday.InRMSE_Range == 1).sum(), "/", df_yesterday.shape[0])

df_yesterday = df_yesterday.drop(['Diff', 'Act_Res', 'Pred_Res'], axis=1).sort_values(f'{tgt_stat}_line', ascending=False)

if df_yesterday.shape[0] >= 50:
    for tm in df_yesterday.Team.unique():
        display(df_yesterday[df_yesterday.Team == tm])
else:
    display(df_yesterday)

RMSE: 4.9285163503604625
Total Accuracy (ParlayHit): 0.544
68 / 125

Total Accuracy (InRMSE_Range): 0.544
68 / 125


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
82,2026-01-02,LAL,Luka Doncic,PG,MEM,35.489624,37.55,34.5,33.136833,34,0.863167,1,1
112,2026-01-02,LAL,LeBron James,SF,MEM,33.395782,36.07,23.5,23.957428,31,7.042572,1,0
109,2026-01-02,LAL,Deandre Ayton,C,MEM,28.358349,24.82,13.5,12.903748,4,8.903748,1,0
61,2026-01-02,LAL,Jake LaRavia,PF,MEM,29.326653,37.03,10.5,12.568539,21,8.431461,1,0
27,2026-01-02,LAL,Marcus Smart,SG,MEM,28.052872,34.28,9.5,12.807664,13,0.192336,1,1
84,2026-01-02,LAL,Jaxson Hayes,C,MEM,17.698044,23.18,5.5,6.854705,12,5.145295,1,0
86,2026-01-02,LAL,Jarred Vanderbilt,PF,MEM,21.536253,20.25,5.5,6.811983,5,1.811983,0,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
9,2026-01-02,OKC,Shai Gilgeous-Alexander,PG,GSW,32.227650,28.18,30.5,25.277224,30,4.722776,1,1
88,2026-01-02,OKC,Jalen Williams,SG,GSW,29.521282,20.55,17.5,16.218920,12,4.218920,1,1
21,2026-01-02,OKC,Chet Holmgren,PF,GSW,28.561832,26.05,17.5,13.693535,15,1.306465,1,1
106,2026-01-02,OKC,Ajay Mitchell,SG,GSW,24.530567,25.08,10.5,9.721630,11,1.278370,0,1
93,2026-01-02,OKC,Luguentz Dort,SF,GSW,24.369114,26.22,7.5,8.655190,11,2.344810,1,1
108,2026-01-02,OKC,Cason Wallace,SG,GSW,26.515209,14.18,7.5,8.099592,2,6.099592,0,0
91,2026-01-02,OKC,Aaron Wiggins,SG,GSW,15.666666,27.08,6.5,5.317776,15,9.682224,0,0
71,2026-01-02,OKC,Isaiah Joe,SG,GSW,15.092714,19.92,6.5,4.730385,5,0.269615,1,1
102,2026-01-02,OKC,Alex Caruso,SG,GSW,15.844961,10.72,4.5,5.334563,4,1.334563,0,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
18,2026-01-02,POR,Deni Avdija,SF,NOP,36.702362,38.40,29.5,25.534489,34,8.465511,0,0
121,2026-01-02,POR,Shaedon Sharpe,SG,NOP,32.794605,30.97,24.5,24.232704,23,1.232704,1,1
62,2026-01-02,POR,Toumani Camara,PF,NOP,33.484592,33.73,15.5,13.441983,14,0.558017,1,1
116,2026-01-02,POR,Donovan Clingan,C,NOP,27.811285,29.52,12.5,12.151600,11,1.151600,1,1
53,2026-01-02,POR,Robert Williams,C,NOP,14.658747,18.48,7.5,5.060315,4,1.060315,1,1
113,2026-01-02,POR,Kris Murray,SF,NOP,26.090050,26.00,6.5,6.928097,6,0.928097,0,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
76,2026-01-02,NYK,Jalen Brunson,PG,ATL,36.017071,34.90,29.5,31.107389,24,7.107389,0,0
104,2026-01-02,NYK,OG Anunoby,PF,ATL,36.382931,34.98,17.5,18.318472,19,0.681528,1,1
70,2026-01-02,NYK,Mikal Bridges,SF,ATL,36.313873,35.15,15.5,13.699465,18,4.300535,0,1
51,2026-01-02,NYK,Miles McBride,SG,ATL,28.126442,30.73,11.5,13.951956,11,2.951956,0,1
65,2026-01-02,NYK,Jordan Clarkson,SG,ATL,22.672504,13.52,10.5,12.440443,5,7.440443,0,0
50,2026-01-02,NYK,Tyler Kolek,PG,ATL,17.651819,17.23,5.5,7.995240,6,1.995240,1,1


...

,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
10,2026-01-02,BRK,Noah Clowney,PF,WAS,30.414898,27.75,19.5,14.603974,8,6.603974,1,0
25,2026-01-02,BRK,Danny Wolf,PF,WAS,23.319443,35.15,13.5,10.042016,11,0.957984,1,1
14,2026-01-02,BRK,Day'Ron Sharpe,C,WAS,18.323725,29.85,12.5,7.953050,14,6.046950,0,0
15,2026-01-02,BRK,Ziaire Williams,SF,WAS,20.905970,23.95,12.5,8.050133,14,5.949867,0,0
97,2026-01-02,BRK,Terance Mann,SG,WAS,27.707796,23.57,8.5,7.555429,14,6.444571,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
87,2026-01-02,SAC,DeMar DeRozan,PF,PHO,32.454983,29.87,18.5,19.805096,13,6.805096,0,0
117,2026-01-02,SAC,Keegan Murray,PF,PHO,33.393211,32.62,14.5,14.841280,23,8.158720,1,0
12,2026-01-02,SAC,Russell Westbrook,PG,PHO,29.594584,26.03,13.5,18.067299,17,1.067299,1,1
42,2026-01-02,SAC,Maxime Raynaud,C,PHO,31.432150,25.42,13.5,16.179531,6,10.179531,0,0
41,2026-01-02,SAC,Dennis Schroder,PG,PHO,27.197021,24.00,11.5,14.186815,12,2.186815,1,1
54,2026-01-02,SAC,Precious Achiuwa,C,PHO,19.998676,18.78,6.5,8.908054,3,5.908054,0,0
4,2026-01-02,SAC,Keon Ellis,SG,PHO,22.681068,34.50,5.5,12.023137,14,1.976863,1,1


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
56,2026-01-02,WAS,CJ McCollum,SG,BRK,33.362247,31.12,18.5,16.238739,17,0.761261,1,1
37,2026-01-02,WAS,Tre Johnson,SG,BRK,24.616859,28.00,14.5,11.658332,12,0.341668,1,1
69,2026-01-02,WAS,Bilal Coulibaly,SG,BRK,28.451031,29.17,10.5,12.337730,11,1.337730,1,1
111,2026-01-02,WAS,Khris Middleton,SF,BRK,22.583683,22.83,9.5,9.992270,9,0.992270,0,1
64,2026-01-02,WAS,Justin Champagnie,SF,BRK,26.316944,24.60,8.5,10.442168,20,9.557832,1,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
85,2026-01-02,CHI,Nikola Vucevic,C,ORL,30.150158,35.18,17.5,16.180603,17,0.819397,1,1
24,2026-01-02,CHI,Ayo Dosunmu,SG,ORL,26.252743,21.15,14.5,10.985415,17,6.014585,0,0
55,2026-01-02,CHI,Matas Buzelis,PF,ORL,26.401369,30.38,14.5,12.122962,21,8.877038,0,0
43,2026-01-02,CHI,Kevin Huerter,SF,ORL,23.234598,23.52,13.5,10.830801,20,9.169199,0,0
29,2026-01-02,CHI,Tre Jones,PG,ORL,25.961311,28.62,13.5,10.304789,6,4.304789,1,1
79,2026-01-02,CHI,Jalen Smith,C,ORL,22.076958,26.95,11.5,9.944106,11,1.055894,1,1
118,2026-01-02,CHI,Isaac Okoro,SG,ORL,23.024878,29.55,9.5,9.837564,9,0.837564,0,1
77,2026-01-02,CHI,Patrick Williams,PF,ORL,16.407253,22.90,7.5,5.905797,15,9.094203,0,0


,Date,Team,Player,Pos,Opp,MP_proj,MP,PTS_line,PTS_proj,PTS,Diff2,ParlayHit,InRMSE_Range
57,2026-01-02,GSW,Brandin Podziemski,SG,OKC,26.520077,26.30,13.5,11.258063,12,0.741937,1,1
22,2026-01-02,GSW,Moses Moody,SG,OKC,20.844461,27.03,12.5,8.694186,13,4.305814,0,1
23,2026-01-02,GSW,Will Richard,SG,OKC,19.106531,27.20,10.5,6.902078,13,6.097922,0,0
40,2026-01-02,GSW,Quinten Post,PF,OKC,16.226662,22.80,9.5,6.733286,11,4.266714,0,1
96,2026-01-02,GSW,Al Horford,C,OKC,15.713541,15.77,5.5,6.510742,13,6.489258,1,0


### Today's predictions

In [27]:
df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)
df_lines = df_lines[~(df_lines.Team.isnull())]

# Predict Mins
df_lines["Team"] = team_encoder.transform(df_lines["Team"])
df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
df_pred_mins = setup_df_mins(con, df_pred)
df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
DM_mins = xgb.DMatrix(df_pred_mins)
df_pred['MP'] = mins_model.predict(DM_mins)

# Predict Stat
df_pred = setup_df_main(df_pred)
feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
df_pred = df_pred[df_pred.Date == now][feature_cols]
DM_stats = xgb.DMatrix(df_pred)
df_pred[f"{tgt_stat}_proj"] = stat_model.predict(DM_stats)

df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
            [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
if tds_picks.shape[0] >= 50:
    print(tds_picks.shape[0], 'rows')
    for tm in tds_picks.Team.unique():
        display(tds_picks[tds_picks.Team == tm])
else:
    display(tds_picks)
tds_picks.insert(0, 'Date', pd.to_datetime(now))
partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

99 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
22,PHI,VJ Edgecombe,SG,NYK,35.871262,34.061343,15.5,23.727512
64,PHI,Paul George,PF,NYK,33.669746,31.700338,15.5,18.067690
7,PHI,Tyrese Maxey,PG,NYK,38.430428,37.430193,28.5,26.284557
35,PHI,Jared McCain,SG,NYK,17.943834,22.231710,5.5,7.251842
23,PHI,Quentin Grimes,SG,NYK,32.756516,31.002632,11.5,13.206612
87,PHI,Joel Embiid,C,NYK,33.295586,30.439123,25.5,26.939037


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
83,NYK,Karl-Anthony Towns,C,PHI,26.868011,32.044260,21.5,14.993954
48,NYK,Mikal Bridges,SF,PHI,35.207760,35.721493,17.5,12.980530
18,NYK,Tyler Kolek,PG,PHI,19.002638,20.766414,6.5,8.480079
29,NYK,Miles McBride,SG,PHI,28.846390,27.388165,12.5,13.389204
6,NYK,Jalen Brunson,PG,PHI,35.544350,36.274335,30.5,30.122810
70,NYK,OG Anunoby,PF,PHI,35.543442,34.181710,18.5,18.667889


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
10,ATL,Trae Young,PG,TOR,26.149031,27.413441,19.5,13.171280
24,ATL,Nickeil Alexander-Walker,SG,TOR,33.434158,31.772491,17.5,23.518175
32,ATL,Dyson Daniels,SG,TOR,34.763962,33.514729,11.5,15.437897
16,ATL,Vit Krejci,PG,TOR,26.140657,23.188073,8.5,11.863371
89,ATL,Onyeka Okongwu,C,TOR,33.855701,32.955739,15.5,18.594761
46,ATL,Jalen Johnson,SF,TOR,36.070961,34.872921,23.5,25.117836
55,ATL,Zaccharie Risacher,SF,TOR,22.816450,22.360861,9.5,10.161245


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
0,UTA,Keyonte George,PG,GSW,34.593086,34.230301,25.5,31.527920
58,UTA,Brice Sensabaugh,SF,GSW,26.898092,22.161047,13.5,17.289040
62,UTA,Lauri Markkanen,PF,GSW,34.672035,34.925885,26.5,25.730770
95,UTA,Jusuf Nurkic,C,GSW,28.820774,28.434407,12.5,12.571485


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
33,DAL,Max Christie,SG,HOU,29.133350,26.867820,9.5,12.702713
51,DAL,Naji Marshall,SF,HOU,28.560669,31.037399,9.5,12.658180
13,DAL,Cooper Flagg,PG,HOU,36.257206,35.492526,19.5,22.108803
65,DAL,P.J. Washington,PF,HOU,32.842449,31.279631,13.5,15.475618
54,DAL,Klay Thompson,SF,HOU,20.010336,21.852258,8.5,9.771058
60,DAL,Anthony Davis,PF,HOU,32.216347,30.577415,21.5,20.598541


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
38,BOS,Jaylen Brown,SF,LAC,34.111588,31.024973,29.5,26.408072
80,BOS,Jordan Walsh,PF,LAC,13.181331,18.782469,4.5,3.507444
26,BOS,Anfernee Simons,SG,LAC,27.191513,26.073606,11.5,12.463745
14,BOS,Payton Pritchard,PG,LAC,33.913990,33.077685,15.5,16.076740
21,BOS,Derrick White,SG,LAC,35.613811,34.227443,18.5,17.974926
100,BOS,Luka Garza,C,LAC,19.311478,17.641428,7.5,7.301432
98,BOS,Neemias Queta,C,LAC,26.410318,26.731059,9.5,9.325795


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
31,CHI,Ayo Dosunmu,SG,CHO,25.150375,24.384123,14.5,11.525713
34,CHI,Isaac Okoro,SG,CHO,25.355804,21.560311,8.5,11.135439
97,CHI,Jalen Smith,C,CHO,22.800617,18.571619,12.5,10.055783
50,CHI,Kevin Huerter,SF,CHO,23.040897,22.264284,13.5,11.913378
63,CHI,Matas Buzelis,PF,CHO,28.281361,25.618005,16.5,17.370329
75,CHI,Patrick Williams,PF,CHO,19.450937,12.854340,9.5,8.705062
88,CHI,Nikola Vucevic,C,CHO,31.846420,28.692606,16.5,15.902207
8,CHI,Tre Jones,PG,CHO,28.309044,25.522522,12.5,12.948923


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
28,SAS,Dylan Harper,SG,POR,22.649738,20.854078,14.5,11.863262
74,SAS,Harrison Barnes,PF,POR,25.501781,26.760782,10.5,8.698776
2,SAS,De'Aaron Fox,PG,POR,32.710930,30.980912,22.5,21.020119
4,SAS,Stephon Castle,PG,POR,32.097675,30.724942,20.5,20.753811
56,SAS,Julian Champagnie,SF,POR,27.740448,25.762982,11.5,11.524851
96,SAS,Luke Kornet,C,POR,23.622192,22.310872,8.5,8.500420


../tables/2025/gmday_preds_PTS.csv saved!
